In [2]:
#Importing the pandas library for reading
import pandas as pd
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [3]:
#Importing the numpy, os, shutil, matplotlib.pyplot
import numpy as np
import os
import shutil
import matplotlib.pyplot as plt

In [4]:
#Importing nltk library for pre processing
import nltk
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud, STOPWORDS
nltk.download('punkt')
nltk.download('wordnet')
import string
nltk.download('stopwords')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
# A dependency of the preprocessing for BERT inputs
#!pip install "tensorflow==2.11.*"
#!pip install -q -U "tensorflow-text==2.11.*"
#!pip install -q _U "tf-models-official==2.11.*"
#!pip install  -q -U tensorflow-hub
#!pip install tensorflow
#!pip install tensorflow-hub
#!pip install tensorflow-text
#!pip install tf-models-official

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 81.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.1/630.1 KB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 KB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 74.8 MB/s eta 0:00:00
   

In [6]:
#!pip install "tensorflow>=2.0.0"
#!pip install --upgrade tensorflow-hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
#Importing sklearn, tensorflow, keras libraries for model implementation
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text


from tensorflow import keras
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras import regularizers

from official.nlp import optimization # to create AdamW optimizer
#from keras.optimizers import optimizer # to create AdamW optimizer

tf.get_logger().setLevel('ERROR')


/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.12.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
train_path = '/content/drive/MyDrive/NLP/data.csv'
df_train = pd.read_csv(train_path)

#Checking the info of the dataset
df_train.info

<bound method DataFrame.info of              id  target                                       comment_text  \
0        239579  0.4400  This is a great story. Man. I wonder if the pe...   
1        239607  0.9125  Yet call out all Muslims for the acts of a few...   
2        239644  0.0000  Because the people who drive cars more are the...   
3        239653  0.3000  Mormons have had a complicated relationship wi...   
4        239744  0.0000                       I'm doing the same thing! :)   
...         ...     ...                                                ...   
235082  6333915  0.3000  Xi and his comrades must be smirking over Trum...   
235083  6333928  0.2000  My thought exactly.  The only people he hasn't...   
235084  6333941  0.0000  I agree, Bill G\nThe vote-buying has begun by ...   
235085  6333947  0.0000  No, the probability of dying may be very, very...   
235086  6333950  0.2000  Nah, I am too boring to parody.  This guy Camp...   

        severe_toxicity  obscen

In [10]:
df_train.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,month,year,Season,Attack,Disability,Religion,Race,Gender,sexual_orientation,Target
0,239579,0.4400,This is a great story. Man. I wonder if the pe...,0,0,0,0,0,0,0,...,1,2016,Summer,NaN,NaN,NaN,NaN,NaN,NaN,0
1,239607,0.9125,Yet call out all Muslims for the acts of a few...,0,0,1,1,0,0,0,...,1,2016,Summer,1.0,NaN,2.0,NaN,NaN,NaN,1
2,239644,0.0000,Because the people who drive cars more are the...,0,0,0,0,0,0,0,...,1,2016,Summer,NaN,NaN,NaN,NaN,NaN,NaN,0
3,239653,0.3000,Mormons have had a complicated relationship wi...,0,0,0,0,0,0,0,...,1,2016,Summer,NaN,NaN,NaN,NaN,NaN,NaN,0
4,239744,0.0000,I'm doing the same thing! :),0,0,0,0,0,0,0,...,1,2016,Summer,NaN,NaN,NaN,NaN,NaN,NaN,0


In [11]:
df_train.fillna(0, inplace=True)
numeric_feature_names = ['comment_text', 'Attack', 'Target']
numeric_features = df_train[numeric_feature_names]
numeric_features.head()

,comment_text,Attack,Target
0,This is a great story. Man. I wonder if the pe...,0.0,0
1,Yet call out all Muslims for the acts of a few...,1.0,1
2,Because the people who drive cars more are the...,0.0,0
3,Mormons have had a complicated relationship wi...,0.0,0
4,I'm doing the same thing! :),0.0,0


In [12]:
#numeric_features.to_csv(r'data2.txt', index=None)

In [13]:
test_path = '/content/drive/MyDrive/NLP/test.csv'
df_test = pd.read_csv(test_path)

#Checking the info of the dataset
df_test.info

<bound method DataFrame.info of             id                                       comment_text
0      7097320  [ Integrity means that you pay your debts.]\n\...
1      7097321  This is malfeasance by the Administrator and t...
2      7097322  @Rmiller101 - Spoken like a true elitist. But ...
3      7097323  Paul: Thank you for your kind words.  I do, in...
4      7097324  Sorry you missed high school. Eisenhower sent ...
...        ...                                                ...
97315  7194635  He should lose his job for promoting mis-infor...
97316  7194636  "Thinning project is meant to lower fire dange...
97317  7194637  I hope you millennials are happy that you put ...
97318  7194638  I'm thinking Kellyanne Conway (a.k.a. The Trum...
97319  7194639  I still can't figure why a pizza in AK cost mo...

[97320 rows x 2 columns]>

In [14]:
stop=set(stopwords.words('english'))

def preprocess_text(text):
      corpus=[]
      #stem=PorterStemmer()
      lem=WordNetLemmatizer()
      for news in text:
          words=[w for w in word_tokenize(news) if (w not in stop)]
          
          words=[lem.lemmatize(w) for w in words if len(w)>2]
          words = [''.join(c for c in s if c not in string.punctuation) for s in words if s]
          words = [word.lower() for word in words]
          words = [word for word in words if word.isalpha()]
          corpus.append(words) 
         
      return corpus     
      
df_train['processed_comment_text']= preprocess_text(df_train['comment_text'])

In [15]:
comment_processed = []
for i in range(len(df_train['processed_comment_text'])):
   comment_processed.append(' '.join(wrd for wrd in df_train.iloc[:,60][i]))
df_train['comment_text_processed'] = comment_processed
df_train.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,Season,Attack,Disability,Religion,Race,Gender,sexual_orientation,Target,processed_comment_text,comment_text_processed
0,239579,0.4400,This is a great story. Man. I wonder if the pe...,0,0,0,0,0,0,0,...,Summer,0.0,0.0,0.0,0.0,0.0,0.0,0,"[this, great, story, man, wonder, person, yell...",this great story man wonder person yelled shut...
1,239607,0.9125,Yet call out all Muslims for the acts of a few...,0,0,1,1,0,0,0,...,Summer,1.0,0.0,2.0,0.0,0.0,0.0,1,"[yet, call, muslims, act, get, pilloried, okay...",yet call muslims act get pilloried okay smear ...
2,239644,0.0000,Because the people who drive cars more are the...,0,0,0,0,0,0,0,...,Summer,0.0,0.0,0.0,0.0,0.0,0.0,0,"[because, people, drive, car, one, cause, wear...",because people drive car one cause wear tear r...
3,239653,0.3000,Mormons have had a complicated relationship wi...,0,0,0,0,0,0,0,...,Summer,0.0,0.0,0.0,0.0,0.0,0.0,0,"[mormons, complicated, relationship, federal, ...",mormons complicated relationship federal law
4,239744,0.0000,I'm doing the same thing! :),0,0,0,0,0,0,0,...,Summer,0.0,0.0,0.0,0.0,0.0,0.0,0,[thing],thing


In [19]:
processed_data = df_train.to_csv(r'processed_data.csv', index = False)

In [16]:
#@title Choose a BERT model to fine-tune

#bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'
#bert_model_name = 'small_bert/bert_en_uncased_L-6_H-768_A-12'
bert_model_name = 'small_bert/bert_en_uncased_L-4_H-768_A-12'



map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [17]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [18]:
classifier_model = build_classifier_model()
bert_raw_result = classifier_model(tf.constant(df_train['comment_text'].head()))
print(tf.sigmoid(bert_raw_result))

KeyboardInterrupt: ignored

In [ ]:
tf.keras.utils.plot_model(classifier_model)

In [ ]:
#Loss Function
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

In [ ]:
#Optimization
epochs = 2
steps_per_epoch = 50 #tf.data.experimental.cardinality(train_data).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [ ]:
#Compile Model
classifier_model.compile(optimizer=optimizer, loss=loss,
                         metrics=metrics)

classifier_model.summary()

In [ ]:
# Split the DataFrame into a text array and a label array
#df_train_x = df_train[['comment_text']]
df_train_x = df_train[['comment_text_processed']]
#df_train_x = df_train[['comment_text_processed', 'Attack']]
df_train_y = df_train[['Target']]

# Define the text vectorizer layer
#vectorizer = TextVectorization(
#    max_tokens=10000,
#    output_mode='int',
#    output_sequence_length=500
#)

# Fit the text vectorizer layer to your text data
#vectorizer.adapt(df_train_x)

# Convert the text data to preprocessed integer sequences
#df_train_x_textvec = vectorizer(df_train_x)

#df_train['text_vec'] = df_train_x_textvec

#df_train_x = df_train[['text_vec', 'comment_text_processed']]

# Apply SMOTE to the text and label data
#smote = SMOTENC(random_state=42, categorical_features=[2])
#smote = SMOTE(random_state=42)
#x_smote, y_smote = smote.fit_resample(df_train_x, df_train_y)

# Split the data into training and testing sets
#X_train4, X_test4, y_train4, y_test4 = train_test_split(text_data_smote, label_data_smote, test_size=0.2, random_state=42)


x_train, x_val, y_train, y_val = train_test_split(df_train_x, df_train_y, 
                                                  test_size=0.2, shuffle = True)

#x_train, x_val, y_train, y_val = train_test_split(x_smote, y_smote, 
#                                                  test_size=0.2, random_state=42)



print('Shape of Training Matrix:'+str(x_train.shape))
print('Shape of Training Target:'+str(y_train.shape))
print('Shape of Validation Matrix:'+str(x_val.shape))
print('Shape of Validation Target:'+str(y_val.shape))

In [ ]:
print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(x = x_train, y = y_train,
                               validation_data = (x_val, y_val),
                               epochs = epochs)

In [ ]:
#Evaluate the model
#df_test_x = df_test[['comment_text']]
y_pred = classifier_model.predict(x = x_val)

In [ ]:
y_pred = np.where(y_pred < 0.5 , 0, 1)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1 score:', f1)

In [ ]:
#Evaluate the model
loss, accuracy = classifier_model.evaluate(x = x_val)

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

In [ ]:
#Plot the accuracy and loss over time
history_dict = history.history
print(history_dict.keys())

acc = history_dict['binary_accuracy']
val_acc = history_dict['val_binary_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)
fig = plt.figure(figsize=(10, 6))
fig.tight_layout()

plt.subplot(2, 1, 1)
# r is for "solid red line"
plt.plot(epochs, loss, 'r', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
# plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')